# Amazon Bedrock AgentCore Runtime で MCP サーバーをホスティング - OAuth インバウンド認証

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime で MCP（Model Context Protocol）サーバーをホスティングする方法を学びます。Amazon Bedrock AgentCore Python SDK を使用して、MCP ツールを Amazon Bedrock AgentCore 互換の MCP サーバーとしてラップします。

Amazon Bedrock AgentCore Python SDK は MCP サーバーの実装詳細を処理するため、ツールのコア機能に集中できます。コードを AgentCore 標準化された MCP プロトコル契約に変換し、直接通信を可能にします。

### チュートリアル詳細

| 情報             | 詳細                                                   |
|:-----------------|:----------------------------------------------------------|
| チュートリアルタイプ | ツールホスティング                                      |
| ツールタイプ      | MCP サーバー                                              |
| チュートリアル構成 | AgentCore Runtime での MCP サーバーホスティング           |
| チュートリアル分野 | クロスバーティカル                                        |
| 例の複雑さ       | 簡単                                                      |
| 使用 SDK        | Amazon BedrockAgentCore Python SDK と MCP                 |

### チュートリアルアーキテクチャ

このチュートリアルでは、MCP サーバーを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、`add_numbers`、`multiply_numbers`、`greet_user` の3つのツールを持つシンプルな MCP サーバーを使用します。

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### チュートリアルの主な機能

* カスタムツールを持つ MCP サーバーの作成
* ローカルでの MCP サーバーのテスト
* Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング
* 認証付きでデプロイされた MCP サーバーの呼び出し

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報の設定
* Amazon Bedrock AgentCore SDK
* MCP（Model Context Protocol）ライブラリ
* Docker が実行中

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# 現在のノートブックのディレクトリを取得
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

In [ ]:
boto_session = Session()
region = boto_session.region_name

ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)

tool_name = "mcp_server_agentcore"

## MCP（Model Context Protocol）の理解

MCP は、AI モデルが外部データやツールに安全にアクセスできるようにするプロトコルです。主な概念：

* **ツール**: AI がアクションを実行するために呼び出せる関数
* **Streamable HTTP**: AgentCore Runtime で使用されるトランスポートプロトコル
* **セッション分離**: 各クライアントは `Mcp-Session-Id` ヘッダーを介して分離されたセッションを取得
* **ステートレス操作**: スケーラビリティのため、サーバーはステートレス操作をサポートする必要あり

AgentCore Runtime は、MCP サーバーがデフォルトパスとして `0.0.0.0:8000/mcp` でホストされることを期待しています。

### プロジェクト構造

適切な構造でプロジェクトをセットアップしましょう：

```
mcp_server_project/
├── mcp_server.py              # メイン MCP サーバーコード
├── my_mcp_client.py          # ローカルテストクライアント
├── my_mcp_client_remote.py   # リモートテストクライアント
├── requirements.txt          # 依存関係
└── __init__.py              # Python パッケージマーカー
```

## MCP サーバーの作成

3つのシンプルなツールを持つ MCP サーバーを作成しましょう。サーバーは AgentCore Runtime 互換性に必要な `stateless_http=True` で FastMCP を使用します。

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """2つの数字を足し算"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """2つの数字を掛け算"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """名前でユーザーに挨拶"""
    return f"こんにちは、{name}さん！お会いできて嬉しいです。"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### このコードの動作

* **FastMCP**: ツールをホストできる MCP サーバーを作成
* **@mcp.tool()**: Python 関数を MCP ツールに変換するデコレーター
* **stateless_http=True**: AgentCore Runtime 互換性に必要
* **ツール**: 異なるタイプの操作を示す3つのシンプルなツール

## ローカルテストクライアントの作成

AgentCore Runtime にデプロイする前に、MCP サーバーをローカルでテストするためのクライアントを作成しましょう：

In [ ]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("利用可能なツール:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### ローカルでのテスト

MCP サーバーをローカルでテストするには：

1. **ターミナル 1**: MCP サーバーを起動
   ```bash
   python mcp_server.py
   ```
   
2. **ターミナル 2**: テストクライアントを実行
   ```bash
   python my_mcp_client.py
   ```

出力に3つのツールがリストされるはずです。

## 認証用の Amazon Cognito のセットアップ

AgentCore Runtime には認証が必要です。Amazon Cognito を使用して、デプロイされた MCP サーバーにアクセスするための JWT トークンを提供します。

In [ ]:
print("Amazon Cognitoユーザープールをセットアップ中...")
cognito_config = setup_cognito_user_pool()
print("Cognitoセットアップ完了 ✓")
print(f"ユーザープールID: {cognito_config.get('pool_id', 'N/A')}")
print(f"クライアントID: {cognito_config.get('client_id', 'N/A')}")

## AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
print(f"使用するAWSリージョン: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"必要なファイル {file} が見つかりません")
print("必要なファイルがすべて見つかりました ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("AgentCore Runtimeを設定中...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("設定完了 ✓")

## MCP サーバーを AgentCore Runtime に起動

Docker ファイルができたので、MCP サーバーを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("MCPサーバーをAgentCore Runtimeに起動中...")
print("数分かかる場合があります...")
launch_result = agentcore_runtime.launch()
print("起動完了 ✓")
print(f"エージェントARN: {launch_result.agent_arn}")
print(f"エージェントID: {launch_result.agent_id}")

## リモートアクセス用の設定の保存

デプロイされた MCP サーバーを呼び出す前に、Agent ARN と Cognito 設定を AWS Systems Manager Parameter Store と AWS Secrets Manager に保存して、簡単に取得できるようにしましょう：

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito認証情報をSecrets Managerに保存しました")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Secrets ManagerのCognito認証情報を更新しました")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ エージェントARNをParameter Storeに保存しました")

print("\n設定の保存が完了しました!")
print(f"エージェントARN: {launch_result.agent_arn}")

## リモートテストクライアントの作成

デプロイされた MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイされたサーバーに接続します：

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta
import traceback

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """リフレッシュトークンを使用してアクセストークンを更新"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """トークンの有効期限を確認し、必要に応じて更新"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 トークンの有効期限が近づいています、更新中...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ トークンを更新しました")
            return new_token
        
        return bearer_token
    except Exception as e:
        print("🔄 無効なトークン、更新中...", e)
        traceback.print_exc()
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"使用するAWSリージョン: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"取得したエージェントARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Secrets Managerから認証情報を取得しました")
        
        # 必要に応じてトークンを検証・更新
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"認証情報取得エラー: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("エラー: AGENT_ARNまたはBEARER_TOKENが正しく取得できませんでした")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\n接続先: {mcp_url}")
    print("ヘッダー設定完了 ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCPセッションを初期化中...")
                await session.initialize()
                print("✓ MCPセッション初期化完了")
                
                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()
                
                print("\n📋 利用可能なMCPツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   説明: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   パラメータ: {list(properties.keys())}")
                    print()
                
                print(f"✅ MCPサーバーへの接続に成功しました!")
                print(f"{len(tool_result.tools)} 個のツールが利用可能です。")
                
    except Exception as e:
        print(f"❌ MCPサーバーへの接続エラー: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## デプロイされた MCP サーバーのテスト

リモートクライアントを使用して、デプロイされた MCP サーバーをテストしましょう：

In [ ]:
print("デプロイされたMCPサーバーをテスト中...")
print("=" * 50)
!python my_mcp_client_remote.py

## MCP ツールをリモートで呼び出す

ツールのリストだけでなく、実際に呼び出して MCP の完全な機能をデモンストレーションする拡張クライアントを作成しましょう：

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """リフレッシュトークンを使用してアクセストークンを更新"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """トークンの有効期限を確認し、必要に応じて更新"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 トークンの有効期限が近づいています、更新中...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ トークンを更新しました")
            return new_token
        
        return bearer_token
    except:
        print("🔄 無効なトークン、更新中...")
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"使用するAWSリージョン: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"取得したエージェントARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Secrets Managerから認証情報を取得しました")
        
        # 必要に応じてトークンを検証・更新
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"認証情報取得エラー: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\n接続先: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCPセッションを初期化中...")
                await session.initialize()
                print("✓ MCPセッション初期化完了")
                
                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()
                
                print("\n📋 利用可能なMCPツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 MCPツールのテスト:")
                print("=" * 50)
                
                try:
                    print("\n➕ add_numbers(5, 3)をテスト中...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   結果: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")
                
                try:
                    print("\n✖️  multiply_numbers(4, 7)をテスト中...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   結果: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")
                
                try:
                    print("\n👋 greet_user('Alice')をテスト中...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   結果: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")
                
                print("\n✅ MCPツールのテストが完了しました!")
                
    except Exception as e:
        print(f"❌ MCPサーバーへの接続エラー: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## ツール呼び出しのテスト

MCP ツールを実際に呼び出してテストしましょう：

In [ ]:
print("MCPツール呼び出しをテスト中...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

MCP サーバーを AgentCore Runtime に正常にデプロイしたので、以下のことができます：

1. **ツールの追加**: MCP サーバーに追加のツールを拡張
2. **カスタム認証**: カスタム JWT 認可を実装
3. **統合**: 他の AgentCore サービスと統合

## クリーンアップ（オプション）

このチュートリアルで作成したリソースをクリーンアップする場合は、以下のセルを実行してください：

In [ ]:
print("🗑️  クリーンアッププロセスを開始中...")

try:
    ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
    print("✓ Parameter Storeのパラメータを削除しました")
except ssm_client.exceptions.ParameterNotFound:
    print("ℹ️  Parameter Storeのパラメータが見つかりません")

try:
    secrets_client.delete_secret(
        SecretId='mcp_server/cognito/credentials',
        ForceDeleteWithoutRecovery=True
    )
    print("✓ Secrets Managerのシークレットを削除しました")
# except secrets_client.exceptions.ResourceNotFoundException:
#     print("ℹ️  Secrets Managerのシークレットが見つかりません")

# print("\n✅ クリーンアップが正常に完了しました!")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 🎉 おめでとうございます！

以下を正常に完了しました：

✅ **MCP サーバーを作成** - カスタムツール付き  
✅ **ローカルでテスト** - MCP クライアントを使用  
✅ **認証をセットアップ** - Amazon Cognito を使用  
✅ **AWS にデプロイ** - AgentCore Runtime を使用  
✅ **リモートで呼び出し** - 適切な認証付き  
✅ **MCP の概念を学習** - ベストプラクティス  

MCP サーバーが Amazon Bedrock AgentCore Runtime で実行され、本番使用の準備が整いました！

## まとめ

このチュートリアルでは、以下を学びました：
- FastMCP を使用した MCP サーバーの構築
- AgentCore 互換性のためのステートレス HTTP トランスポートの設定
- Amazon Cognito を使用した JWT 認証のセットアップ
- AWS への MCP サーバーのデプロイと管理
- ローカルとリモートの両方でのテスト
- ツール呼び出しのための MCP クライアントの使用

デプロイされた MCP サーバーは、より大きな AI アプリケーションやワークフローに統合できるようになりました！